
 # Advanced Lane Finding Project

The steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.


## Start Project Code Here

### Camera calibration function

In [1]:
# Camera calibration

import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib qt

# Camera calibration

def camera_calibration():
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')

    # replace the double backslash, windows doesn't like it
    for index in range(len(images)):
        images[index] = images[index].replace('\\', '/')


    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    
    return mtx, dist
   

In [2]:
# Undistortion

def img_undistort(img, mtx, dist) :
    img_undistort = cv2.undistort(img, mtx, dist, None, mtx)
    
    return img_undistort


### Color and gradient threshold code function

In [3]:
# Color and gradient threshold
# code from lesson

import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg


def pipeline(img, s_thresh=(70, 255), sx_thresh=(20, 100)):
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]

    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    # Note color_binary[:, :, 0] is all 0s, effectively an all black image. It might
    # be beneficial to replace this channel with something else.
    color_binary = np.zeros_like(sxbinary)
    #color_binary = np.dstack(( color_binary, sxbinary, s_binary)) * 255
    color_binary[(s_binary == 1) | (sxbinary == 1)] = 1

    # region of interest
    roi = np.array([[[600,400],[700,400],[1180,720],[100,720]]], np.int32)
    mask = np.zeros_like(color_binary)
    if len(color_binary.shape) > 2:
        channel_count = color_binary.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255

    cv2.fillPoly(mask, roi, ignore_mask_color)
    color_binary = cv2.bitwise_and(color_binary, mask)

    
    return color_binary



### Warping function

In [4]:
# code pasted from discussions
# warps an image

# Destination points are chosen such that straight lanes appear more or less parallel in the transformed image.
dest_bottom_left = [320,720] 
dest_bottom_right = [920, 720]
dest_top_left = [320, 1]
dest_top_right = [920, 1]
src_bottom_left = [200,720]
src_bottom_right = [1124,720]
src_top_left = [577,456]
src_top_right = [710,456]

def warp_image(img):
    source = np.float32([ src_top_left, src_top_right, src_bottom_right, src_bottom_left ])
    dst = np.float32([dest_top_left, dest_top_right, dest_bottom_right, dest_bottom_left])
    M = cv2.getPerspectiveTransform(source, dst)
    #M_inv = cv2.getPerspectiveTransform(dst, source)
    img_size = img.shape[1::-1]

    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return warped

def get_unwarp_mtx():
    source = np.float32([ src_top_left, src_top_right, src_bottom_right, src_bottom_left ])
    dst = np.float32([dest_top_left, dest_top_right, dest_bottom_right, dest_bottom_left])
    return cv2.getPerspectiveTransform(dst, source)
    


### Sliding Windows function

In [5]:
# initial sliding windows code
# pasted from lesson

import numpy as np
import cv2
import matplotlib.pyplot as plt

def create_lane_line_windows(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and  visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    return out_img, left_fit, right_fit, left_lane_inds, right_lane_inds, nonzerox, nonzeroy



In [6]:
# alternate sliding windows

import numpy as np
import cv2
import matplotlib.pyplot as plt

def create_lane_line_windows_alt1(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set height of windows
    window_height = np.int(binary_warped.shape[0]/nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated for each window
    leftx_current = leftx_base
    rightx_current = rightx_base
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []
    # Create empty lists to hold window corners
    left_rects = []
    right_rects = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Append the corners to the corners list
        left_rects.append([(win_xleft_low,win_y_low),(win_xleft_high,win_y_high)])
        right_rects.append([(win_xright_low,win_y_low),(win_xright_high,win_y_high)])
        # Identify the nonzero pixels in x and y within the window
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)

    return left_lane_inds, right_lane_inds, left_rects, right_rects



In [7]:
# Find windows after a successful previous call
# Assume you now have a new warped binary image 
# from the next frame of video (also called "binary_warped")
# It's now much easier to find line pixels!
def create_lane_line_windows2(binary_warped, left_fit, right_fit):
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    margin = 100
    left_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + 
        left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + 
        left_fit[1]*nonzeroy + left_fit[2] + margin))) 

    right_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + 
        right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + 
        right_fit[1]*nonzeroy + right_fit[2] + margin)))  

   
    return left_inds, right_inds
    


## Visualizations and functions for single images/frames here

In [14]:
# 0a Chessboard distortion/undistortion
#
#
#
#

fname = 'calibration1.jpg'
image = mpimg.imread('camera_cal/' + fname)
print(image.shape)

mtx, dist = camera_calibration()

undist_image = img_undistort(image, mtx, dist)

# debug below
# visualization
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 9))
ax1.imshow(image)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(undist_image, cmap='gray')
ax2.set_title('Undistorted Image', fontsize=30)

plt.savefig('output_images/undistorted_' + fname)

(720, 1280, 3)


In [11]:
# 0b Chessboard distortion/undistortion
#
#
#
#

fname = 'straight_lines1.jpg'
image = mpimg.imread('test_images/' + fname)
print(image.shape)

mtx, dist = camera_calibration()

undist_image = img_undistort(image, mtx, dist)

# debug below
# visualization
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 9))
ax1.imshow(image)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(undist_image, cmap='gray')
ax2.set_title('Undistorted Image', fontsize=30)

plt.savefig('output_images/undistorted_' + fname)

(720, 1280, 3)


In [13]:
# 1 Color/gradient visualization
#
#
#
#

fname = 'straight_lines1.jpg'
image = mpimg.imread('test_images/' + fname)
print(image.shape)

binary_image = pipeline(undist_image, (170, 255))

# debug below
# visualization
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 9))
ax1.imshow(image)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(binary_image, cmap='gray')
ax2.set_title('Binary Image', fontsize=30)

plt.savefig('output_images/binary_' + fname)

(720, 1280, 3)


In [14]:
# 2 Warp function visualization
#
#
#
#

src_bottom_left = [200,720]
src_bottom_right = [1124,720]
src_top_left = [577,456]
src_top_right = [710,456]
dest_bottom_left = [320,720] 
dest_bottom_right = [920, 720]
dest_top_left = [320, 1]
dest_top_right = [920, 1]


binary_warped = warp_image(binary_image)

f, (ax1, ax2) = plt.subplots(1, 2, figsize=(15,9))
f.tight_layout()
ax1.imshow(image)
ax1.plot([200, 1124], [720, 720], color='red',linewidth=2)
ax1.plot([1124, 710], [720,456], color='red', linewidth=2)
ax1.plot([710,577], [456,456], color='red', linewidth=2)
ax1.plot([577,200],[456,720], color='red', linewidth=2)
ax1.set_title('Original Image', fontsize=20)
ax2.imshow(binary_warped, cmap='gray')
ax2.plot([320,920], [720, 720], color='red',linewidth=2)
ax2.plot([920,920], [720,1], color='red', linewidth=2)
ax2.plot([920,320], [1,1], color='red', linewidth=2)
ax2.plot([320,320],[1,720], color='red', linewidth=2)
ax2.set_title('Warped Binary Image', fontsize=20)
#plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)

plt.savefig('output_images/binary_warped_' + fname)

In [15]:
# 3 Sliding windows first function visualization
#
#
# 
# 

out_img, left_fit, right_fit, left_inds, right_inds, nonzerox, nonzeroy = create_lane_line_windows(binary_warped)

ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

out_img[nonzeroy[left_inds], nonzerox[left_inds]] = [255, 0, 0]
out_img[nonzeroy[right_inds], nonzerox[right_inds]] = [0, 0, 255]

plt.imshow(out_img)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 1280)
plt.ylim(720, 0)

#plt.savefig('output_images/line_fits_' + fname)

(720, 0)

In [16]:
# 4 sliding windows alt1 function visualization 
# 
# 
#
#

left_inds, right_inds, left_rects, right_rects = create_lane_line_windows_alt1(binary_warped)

out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255

nonzero = binary_warped.nonzero()
nonzeroy = np.array(nonzero[0])
nonzerox = np.array(nonzero[1])

# Extract left and right line pixel positions
leftx = nonzerox[left_inds]
lefty = nonzeroy[left_inds] 
rightx = nonzerox[right_inds]
righty = nonzeroy[right_inds] 

# Fit a second order polynomial to each
left_fit = np.polyfit(lefty, leftx, 2)
right_fit = np.polyfit(righty, rightx, 2)


ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

out_img[nonzeroy[left_inds], nonzerox[left_inds]] = [255, 0, 0]
out_img[nonzeroy[right_inds], nonzerox[right_inds]] = [0, 0, 255]

for i in range(9):
    # Draw the rectangles (sliding windows) on the visualization image
    cv2.rectangle(out_img,left_rects[i][0],left_rects[i][1],(0,255,0), 2) 
    cv2.rectangle(out_img,right_rects[i][0],right_rects[i][1],(0,255,0), 2) 


plt.imshow(out_img)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 1280)
plt.ylim(720, 0)

plt.savefig('output_images/line_fits_' + fname)

In [17]:
# 5 Find radius of curvature
# Fits lines to the curve
# pasted from lesson
#
#

# Y value used for calculating
y_eval = np.max(ploty)

# make temp color image
temp_img = np.dstack((binary_warped, binary_warped, binary_warped))*255


# Define y-value where we want radius of curvature
ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]


plt.imshow(temp_img)
plt.plot(left_fitx, ploty, color='red')
plt.plot(right_fitx, ploty, color='blue')
plt.xlim(0, 1280)
plt.ylim(720, 0)

plt.savefig('output_images/warped_line_fits_' + fname)

"""
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 9))
ax2.set_title('Warped with Fits', fontsize=30)
ax2.imshow(temp_img)
ax2.plot(left_fitx, ploty, 'o', color='red', linewidth=2)
ax2.plot(right_fitx, ploty, 'o', color='blue', linewidth=2)

ax1.set_title('Unwarped with Fits', fontsize=30)
M_inv = get_unwarp_mtx()
newwarp = cv2.warpPerspective(temp_img, M_inv, (image.shape[1], image.shape[0])) 
ax1.imshow(newwarp)
ax1.plot(left_fitx, ploty, 'o', color='red', linewidth=2)
ax1.plot(right_fitx, ploty, 'o', color='blue', linewidth=2)
"""

left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
print(left_curverad, right_curverad)

# Define conversions in x and y from pixels space to meters
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension

# Fit new polynomials to x,y in world space
left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
# Calculate the new radii of curvature
left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
# Now our radius of curvature is in meters
print(left_curverad, 'm', right_curverad, 'm')



84591.0628787 110252.904967
27623.9603911 m 35883.9107241 m


In [20]:
# 6 visualization
# Create an image to draw on and an image to show the selection window
#
#
#

out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
window_img = np.zeros_like(out_img)

_, left_fit, right_fit, left_lane_inds, right_lane_inds, nonzerox, nonzeroy = create_lane_line_windows(binary_warped)
#left_fit, right_fit, left_inds, right_inds, nonzerox, nonzeroy = create_lane_line_windows2(binary_warped, left_fit, right_fit)


ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

margin = 100

# Color in left and right line pixels
out_img[nonzeroy[left_inds], nonzerox[left_inds]] = [255, 0, 0]
out_img[nonzeroy[right_inds], nonzerox[right_inds]] = [0, 0, 255]

# Generate a polygon to illustrate the search window area
# And recast the x and y points into usable format for cv2.fillPoly()
left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, ploty])))])
left_line_pts = np.hstack((left_line_window1, left_line_window2))
right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, ploty])))])
right_line_pts = np.hstack((right_line_window1, right_line_window2))

# Draw the lane onto the warped blank image
cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
plt.imshow(result)
plt.plot(left_fitx, ploty, color='yellow')
plt.plot(right_fitx, ploty, color='yellow')
plt.xlim(0, 1280)
plt.ylim(720, 0)

(720, 0)

In [19]:
# 7 Drawing the final image
# pasted from lesson
#
#
#

# Create an image to draw the lines on
warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

# Recast the x and y points into usable format for cv2.fillPoly()
pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
pts = np.hstack((pts_left, pts_right))

# Draw the lane onto the warped blank image
cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

# Warp the blank back to original image space using inverse perspective matrix (Minv)
M_inv = get_unwarp_mtx()
newwarp = cv2.warpPerspective(color_warp, M_inv, (image.shape[1], image.shape[0])) 
# Combine the result with the original image
result = cv2.addWeighted(image, 1, newwarp, 0.3, 0)

# Text for curve and offset
# Calculate curvature and offset
font = cv2.FONT_HERSHEY_SIMPLEX

# Y value used for calculating
y_eval = np.max(ploty)

# Define conversions in x and y from pixels space to meters
ym_per_pix = 30/720 # meters per pixel in y dimension
xm_per_pix = 3.7/700 # meters per pixel in x dimension

# Fit new polynomials to x,y in world space
left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
# Calculate the new radii of curvature
left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])

# Add text to an image
topline_text = 'Curvature: L %.3fm  R %.3fm' % (left_curverad, right_curverad)

# how to calculate camera offset
camera_position = image.shape[1]/2
lane_center = (right_fitx[719] + left_fitx[719])/2
center_offset_pixels = camera_position - lane_center
center_offset_dist = center_offset_pixels * xm_per_pix

bottomline_text = 'Offset: %.3fm' % (center_offset_dist)

cv2.putText(result,topline_text,(200,50), font, 1,(255,255,255),2,cv2.LINE_AA)
cv2.putText(result,bottomline_text,(200,100), font, 1, (255,255,255),2,cv2.LINE_AA)


plt.imshow(result)
plt.savefig('output_images/final_' + fname)

### Video packages and routines

In [8]:
# Get matrix and info for camera calibration
mtx, dist = camera_calibration()

In [9]:

# Text for curvature and offset
topline_text = ''
bottomline_text = ''


# main function for processing images for video
def process_video_image(image):
    # Assume camera calibration has been done already
    # Declare global var for number of frames
    global num_frames
    global topline_text
    global bottomline_text
    
    # various globals for smoothing
    global all_left_fits, all_right_fits
    global last_left_pts, last_right_pts
    
    # Undistort original camera image
    undis_image = img_undistort(image, mtx, dist)
    
    # Color and gradient filters
    binary_image = pipeline(undis_image, (170, 255))
    
    # warped binary image
    binary_warped = warp_image(binary_image)

    # Uses alt1 image to fit lane lines and find sliding windows
    #
    """
    left_inds = []
    right_inds = []
    if len(last_left_pts) and len(last_right_pts):
        left_inds, right_inds = create_lane_line_windows2(binary_warped, all_left_fits[-1], all_right_fits[-1])
    else :
        left_inds, right_inds, _, _ = create_lane_line_windows_alt1(binary_warped)
    """

    
    left_inds, right_inds, _, _ = create_lane_line_windows_alt1(binary_warped)

    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    # Extract left and right line pixel positions
    leftx = nonzerox[left_inds]
    lefty = nonzeroy[left_inds] 
    rightx = nonzerox[right_inds]
    righty = nonzeroy[right_inds] 

    # Fit a second order polynomial to each
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    all_left_fits.append(left_fit)
    all_right_fits.append(right_fit)
    
    # Plot the lane line according to the fit
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    # sanity checks
    # smoothing attempts
    # get running avg of last 3 top lane widths
    # best check seems to be comparing top pixel position with running average
    avg_left_pt = avg_right_pt = 0.
    if(len(last_left_pts) > 3):
        avg_left_pt  = (last_left_pts[-3][0] + last_left_pts[-2][0] + last_left_pts[-1][0])/3
        avg_right_pt = (last_right_pts[-3][0] + last_right_pts[-2][0] + last_right_pts[-1][0])/3
        
        avg_width = avg_right_pt - avg_left_pt

        margin = 50
        if left_fitx[0] < avg_left_pt - margin or left_fitx[0] > avg_left_pt + margin :
            left_fitx = last_left_pts[-1]
        if right_fitx[0] < avg_right_pt - margin or right_fitx[0] > avg_right_pt + margin :
            right_fitx = last_right_pts[-1]
                    
    last_left_pts.append(left_fitx)
    last_right_pts.append(right_fitx)
    
    
    # Create an image to draw the lines on
    #
    #
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    M_inv = get_unwarp_mtx()
    newwarp = cv2.warpPerspective(color_warp, M_inv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(image, 1, newwarp, 0.3, 0)
    #
    #
    #
    
    # Calculate curvature and offset every n frames
    font = cv2.FONT_HERSHEY_SIMPLEX

    # Y value used for calculating
    y_eval = np.max(ploty)

    # Define conversions in x and y from pixels space to meters
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/600 # meters per pixel in x dimension

    # Fit new polynomials to x,y in world space
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    # Calculate the new radii of curvature
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])

    # Add text to an image
    topline_text = 'Curvature: L %.3fm  R %.3fm' % (left_curverad, right_curverad)

    # how to calculate camera offset
    camera_position = image.shape[1]/2
    lane_center = (right_fitx[719] + left_fitx[719])/2
    center_offset_pixels = camera_position - lane_center
    center_offset_dist = center_offset_pixels * xm_per_pix

    bottomline_text = 'Offset: %.3fm' % (center_offset_dist)

    cv2.putText(result,topline_text,(200,50), font, 1,(255,255,255),2,cv2.LINE_AA)
    cv2.putText(result,bottomline_text,(200,100), font, 1, (255,255,255),2,cv2.LINE_AA)
    
    """
    # debug and smoothing info
    temp_text = 'Frame: %.0f' % (num_frames)
    cv2.putText(result,temp_text,(10,150), font, 1, (255,255,255),2,cv2.LINE_AA)
    
    if not bad_fit:
        cv2.putText(result,bad_fit,(10,200), font, 1, (255,0,0),2,cv2.LINE_AA)
    else:
        cv2.putText(result,'Good Fit',(10,200), font, 1, (255,255,255),2,cv2.LINE_AA)
    temp_text = '    Left fit: %.9f %.3f %.3f' % (left_fit[0], left_fit[1], left_fit[2])
    cv2.putText(result,temp_text,(10,250), font, 1, (255,255,255),2,cv2.LINE_AA)
    temp_text = 'Avg Left fit: {:8.7f}     {:04.3f} {:04.3f}'.format(avg_left_fit[0], avg_left_fit[1], avg_left_fit[2])
    cv2.putText(result,temp_text,(10,300), font, 1, (255,255,255),2,cv2.LINE_AA)
    temp_text = '    Right fit: %.9f %.3f %.3f' % (right_fit[0], right_fit[1], right_fit[2])
    cv2.putText(result,temp_text,(10,350), font, 1, (255,255,255),2,cv2.LINE_AA)
    temp_text = 'Avg Right fit: {:8.7f}     {:04.3f} {:04.3f}'.format(avg_right_fit[0], avg_right_fit[1], avg_right_fit[2])
    cv2.putText(result,temp_text,(10,400), font, 1, (255,255,255),2,cv2.LINE_AA)
    
    
    debugfile = open('debug1.csv','a')
    debugfile.write('{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n'.format(left_fit[0], left_fit[1], 
                                        left_fit[2], right_fit[0], right_fit[1], right_fit[2],
                                        avg_left_fit[0], avg_left_fit[1], avg_left_fit[2],
                                        avg_right_fit[0], avg_right_fit[1], avg_right_fit[2],
                                        left_curverad, right_curverad, left_fitx[0], right_fitx[0],
                                        left_fitx[-1], right_fitx[-1], bad_fit))
    debugfile.close()
    """
    
    
    
    num_frames += 1
    
    return result





In [10]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

num_frames = 0
# To keep track of fits
all_left_fits = []
all_right_fits = []
last_left_pts = []
last_right_pts = []




white_output = 'project_video_output.mp4'
##clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4").subclip(0,5)
clip1 = VideoFileClip('project_video.mp4')
white_clip = clip1.fl_image(process_video_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)


[MoviePy] >>>> Building video project_video_output.mp4
[MoviePy] Writing video project_video_output.mp4


100%|█████████████████████████████████████▉| 1260/1261 [02:12<00:00,  9.24it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_output.mp4 

Wall time: 2min 13s


In [16]:
# Writing individual frames to disk

vidcap = cv2.VideoCapture('project_video_output.mp4')
success,image = vidcap.read()
count = 0
success = True
while success:
  success,image = vidcap.read()
  print('Read a new frame: ', success)
  cv2.imwrite('video_frames/frame%d.jpg' % count, image)     # save frame as JPEG file
  count += 1

Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame:  True
Read a new frame

### Debug code testing different threshold values

In [22]:
# testing and debug
# adjust threshold with sliders

from ipywidgets import widgets
from ipywidgets import interact

image = mpimg.imread('test_images/straight_lines1.jpg')

def f(x, y):
    #color_binary = pipeline(image, s_thresh=(x, y))
    color_binary = pipeline(image, (70,255), sx_thresh=(x,y))
    f, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 9))
    ax1.imshow(image)
    ax1.set_title('Original Image', fontsize=30)
    ax2.imshow(color_binary, cmap='gray')
    ax2.set_title('Binary Image', fontsize=30)
    
    
    
    
    
interact(f, x=widgets.IntSlider(min=0,max=255,step=1,
                                value=170, continuous_update=False, description="Min Color:"),
                                y=widgets.IntSlider(min=0,max=255,step=1,
                                value=255, continuous_update=False, description="Max Color:"))

<function __main__.f>

### Convolution method of line detection

In [ ]:
#  Pasted from lesson

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob
import cv2

# Read in a thresholded image
#warped = mpimg.imread('examples/warped_example.jpg')
warped = bin_result
# window settings
window_width = 50 
window_height = 80 # Break image into 9 vertical layers since image height is 720
margin = 100 # How much to slide left and right for searching

def window_mask(width, height, img_ref, center,level):
    output = np.zeros_like(img_ref)
    output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
    return output

def find_window_centroids(image, window_width, window_height, margin):
    
    window_centroids = [] # Store the (left,right) window centroid positions per level
    window = np.ones(window_width) # Create our window template that we will use for convolutions
    
    # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
    # and then np.convolve the vertical image slice with the window template 
    
    # Sum quarter bottom of image to get slice, could use a different ratio
    l_sum = np.sum(image[int(3*image.shape[0]/4):,:int(image.shape[1]/2)], axis=0)
    l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
    r_sum = np.sum(image[int(3*image.shape[0]/4):,int(image.shape[1]/2):], axis=0)
    r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(image.shape[1]/2)
    
    # Add what we found for the first layer
    window_centroids.append((l_center,r_center))
    
    # Go through each layer looking for max pixel locations
    for level in range(1,(int)(image.shape[0]/window_height)):
        # convolve the window into the vertical slice of the image
        image_layer = np.sum(image[int(image.shape[0]-(level+1)*window_height):int(image.shape[0]-level*window_height),:], axis=0)
        conv_signal = np.convolve(window, image_layer)
        # Find the best left centroid by using past left center as a reference
        # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
        offset = window_width/2
        l_min_index = int(max(l_center+offset-margin,0))
        l_max_index = int(min(l_center+offset+margin,image.shape[1]))
        l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
        # Find the best right centroid by using past right center as a reference
        r_min_index = int(max(r_center+offset-margin,0))
        r_max_index = int(min(r_center+offset+margin,image.shape[1]))
        r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
        # Add what we found for that layer
        window_centroids.append((l_center,r_center))

    return window_centroids

window_centroids = find_window_centroids(warped, window_width, window_height, margin)

# If we found any window centers
if len(window_centroids) > 0:

    # Points used to draw all the left and right windows
    l_points = np.zeros_like(warped)
    r_points = np.zeros_like(warped)

    # Go through each level and draw the windows 
    for level in range(0,len(window_centroids)):
        # Window_mask is a function to draw window areas
        l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
        r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
        # Add graphic points from window mask here to total pixels found 
        l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
        r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

    # Draw the results
    template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
    zero_channel = np.zeros_like(template) # create a zero color channel
    template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
    warpage= np.dstack((warped, warped, warped))*255 # making the original road pixels 3 color channels
    output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results
 
# If no window centers found, just display original road image
else:
    output = np.array(cv2.merge((warped,warped,warped)),np.uint8)

# Display the final results
plt.imshow(output)
plt.title('window fitting results')
plt.show()


In [ ]:
# Fits dots and lines for drawing curvature
# code pasted from lesson
# currently uses generated data

import numpy as np
import matplotlib.pyplot as plt
# Generate some fake data to represent lane-line pixels
ploty = np.linspace(0, 719, num=720)# to cover same y-range as image
quadratic_coeff = 3e-4 # arbitrary quadratic coefficient
# For each y position generate random x position within +/-50 pix
# of the line base position in each case (x=200 for left, and x=900 for right)
leftx = np.array([200 + (y**2)*quadratic_coeff + np.random.randint(-50, high=51) 
                              for y in ploty])
rightx = np.array([900 + (y**2)*quadratic_coeff + np.random.randint(-50, high=51) 
                                for y in ploty])

leftx = leftx[::-1]  # Reverse to match top-to-bottom in y
rightx = rightx[::-1]  # Reverse to match top-to-bottom in y


# Fit a second order polynomial to pixel positions in each fake lane line
left_fit = np.polyfit(ploty, leftx, 2)
left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
right_fit = np.polyfit(ploty, rightx, 2)
right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

# Plot up the fake data
mark_size = 3
plt.plot(leftx, ploty, 'o', color='red', markersize=mark_size)
plt.plot(rightx, ploty, 'o', color='blue', markersize=mark_size)
plt.xlim(0, 1280)
plt.ylim(0, 720)
plt.plot(left_fitx, ploty, color='green', linewidth=3)
plt.plot(right_fitx, ploty, color='green', linewidth=3)
plt.gca().invert_yaxis() # to visualize as we do the images


### Line Class

In [ ]:
# Code for the line class pasted from lesson

# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
